In [1]:
import matplotlib.pylab as plt
from sklearn.metrics import classification_report

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalMaxPooling2D
from keras.optimizers import Adam

import tensorflow as tf
from keras import backend as K
from keras import initializers

import numpy as np
import random

Using TensorFlow backend.


In [2]:
from utils_ import *
from data_ import *

In [11]:
array_1 = [0,1,2,3,4]
array_2 = [5,6,7,8,9]

array_1,array_2 = random.shuffle((array_1,array_2))

print(array_1)
print("\n")
print(array_2)


TypeError: 'tuple' object does not support item assignment

In [10]:
random.seed(42)
np.random.seed(42)
tf.set_random_seed(42)
my_init = initializers.glorot_uniform(seed=42)

In [5]:
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [6]:
points = load_keypoints_txt_from_folders("./facs_me_db/Landmarks/")
images = load_photos_from_folders("./facs_me_db/cohn-kanade-images/")
labels = load_label_txt_from_folders("./facs_me_db/Emotion/")
labels2 = load_label_txt_from_folders("./facs_me_db/FACS_labels/FACS/")

In [7]:
X, Y1, Y2, Y3 = make_x_y(points, images, labels, labels2)

In [8]:
X_train, X_test, Y_train, Y_test, Y_train2, Y_test2, Y_train3, Y_test3 = prepare_train_test(X, Y1, Y2, Y3, W)
# 1: emotions
# 2: FACS
# 3: facial keypoints

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [0]:
input_shape = (W, W, 1)

# Plain CNN with a single output

In [0]:
visible = Input(shape=input_shape)
x = Conv2D(10, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(visible)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(20, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(30, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = GlobalMaxPooling2D()(x)
output = Dense(Y_train.shape[1], activation='softmax', kernel_initializer=my_init)(x)
model = Model(inputs=visible, outputs=output)

In [0]:
model.summary()

In [0]:
model.compile(loss=['categorical_crossentropy'],
              optimizer=Adam(clipnorm = 1.), metrics = ['accuracy'])

In [0]:
model.fit(X_train, Y_train,
          validation_data = (X_test, Y_test),
          batch_size = 64,
          epochs = 10,
          verbose = True,
          shuffle = False)

In [0]:
pred = model.predict(X_test)
y_pred = [np.argmax(p) for p in pred]
y_true = [np.argmax(p) for p in Y_test]

print classification_report(y_true, y_pred)

# CNN with auxiliary FACS output

In [0]:
visible = Input(shape=input_shape)
x = Conv2D(10, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(visible)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(20, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(30, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = GlobalMaxPooling2D()(x)
output = Dense(Y_train.shape[1], activation='softmax', name = 'emotion',
               kernel_initializer=my_init)(x)
output2 = Dense(Y_train2.shape[1], activation='sigmoid', name = 'facs',
                kernel_initializer=my_init)(x)
model = Model(inputs=visible, outputs=[output, output2])

In [0]:
model.summary()

In [0]:
model.compile(loss=['categorical_crossentropy', 'binary_crossentropy'],
              optimizer=Adam(clipnorm = 1.), metrics = {'emotion': 'accuracy'}, loss_weights = [1, 1])

In [0]:
model.fit(X_train, [Y_train, Y_train2],
          validation_data = (X_test, [Y_test, Y_test2]),
          batch_size = 64,
          epochs = 10,
          verbose = True,
          shuffle = False)

In [0]:
pred = model.predict(X_test)[0]
y_pred = [np.argmax(p) for p in pred]
y_true = [np.argmax(p) for p in Y_test]

print classification_report(y_true, y_pred)

# CNN with auxiliary KEYPOINT output

In [0]:
visible = Input(shape=input_shape)
x = Conv2D(10, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(visible)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(20, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(30, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x)
x = GlobalMaxPooling2D()(x)
output = Dense(Y_train.shape[1], activation='softmax', name = 'emotion', kernel_initializer=my_init)(x)
output2 = Dense(Y_train3.shape[1], activation='linear', kernel_initializer=my_init)(x)
model = Model(inputs=visible, outputs=[output, output2])

In [0]:
model.summary()

In [0]:
model.compile(loss=['categorical_crossentropy', 'mse'],
              optimizer=Adam(clipnorm = 1.), metrics = {'emotion': 'accuracy'}, loss_weights = [1, 1])

In [0]:
model.fit(X_train, [Y_train, Y_train3],
          validation_data = (X_test, [Y_test, Y_test3]),
          batch_size = 64,
          epochs = 10,
          verbose = True,
          shuffle = False)

In [0]:
pred = model.predict(X_test)[0]
y_pred = [np.argmax(p) for p in pred]
y_true = [np.argmax(p) for p in Y_test]

print classification_report(y_true, y_pred)

# CNN with  FACS and KEYPOINT

In [0]:
visible = Input(shape=input_shape)
x1 = Conv2D(10, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(visible)
x1 = MaxPooling2D(pool_size=(2, 2))(x1)
x2 = Conv2D(20, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x1)
x2 = MaxPooling2D(pool_size=(2, 2))(x2)
x3 = Conv2D(30, kernel_size=3, activation='elu', padding='same', kernel_initializer=my_init)(x2)
x = GlobalMaxPooling2D()(x3)
output = Dense(Y_train.shape[1], activation='softmax', name = 'emotion', kernel_initializer=my_init)(x)
output2 = Dense(Y_train2.shape[1], activation='sigmoid', kernel_initializer=my_init)(GlobalMaxPooling2D()(x2))
output3 = Dense(Y_train3.shape[1], activation='linear', kernel_initializer=my_init)(GlobalMaxPooling2D()(x1))
model = Model(inputs=visible, outputs=[output, output2, output3])

In [0]:
model.summary()

In [0]:
model.compile(loss=['categorical_crossentropy', 'binary_crossentropy', 'mse'],
              optimizer=Adam(clipnorm = 1.), metrics = {'emotion': 'accuracy'}, loss_weights = [1, 1e-1, 1e-1])

In [0]:
model.fit(X_train, [Y_train, Y_train2, Y_train3],
          validation_data = (X_test, [Y_test, Y_test2, Y_test3]),
          batch_size = 64,
          epochs = 10,
          verbose = True,
          shuffle = False)

In [0]:
pred = model.predict(X_test)[0]
y_pred = [np.argmax(p) for p in pred]
y_true = [np.argmax(p) for p in Y_test]

print classification_report(y_true, y_pred)

In [0]:
# homework: to get 65% F1